In [1]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime
from playsound import playsound
from numpy import nan
import pandas as pd
import numpy
# from PIL import ImageGrab
from datetime import timedelta
import pymysql

In [2]:
##########################################Function to add encodings to csv file############################################

import face_recognition
import pandas 
import cv2
def DbEncodings(img_dir,Name):
    '''img_dir = image file name with extension
        Name = Name of the person in the image'''
    encList = []
    images = cv2.imread(img_dir)
    image = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
    try:
        enc = face_recognition.face_encodings(image)[0]
    except IndexError as e:
        print(e)
    #encList.append(enc)
    print(enc)
    encod = pd.read_csv('encodingsKnown.csv')
    #enc = np.array(enc)
    print(enc)
    Name = Name.upper()
    print(encod.head())
    encod.loc[len(encod.index)]=[Name,enc]
    encod.to_csv('encodingsKnown.csv',index=False)
    return Name
current_in = {}

In [3]:
DbEncodings("Abdul Samad.jpg","Abdul Samad")

[-0.06873904  0.04569402 -0.01135144 -0.06748459 -0.05979402  0.05479646
 -0.03561673 -0.11821745  0.16538602 -0.16614649  0.23596525 -0.06240964
 -0.17839013 -0.08938353  0.02108858  0.07190797 -0.0561984  -0.18411386
 -0.008893   -0.0821109   0.01041657 -0.06469677  0.01720974  0.06863295
 -0.2191499  -0.35474423 -0.10379234 -0.06736074  0.01953099 -0.11306845
 -0.01688067  0.00155357 -0.24547815 -0.03170744 -0.0479262   0.14580807
 -0.04777739 -0.04498594  0.21553485  0.05471691 -0.11671313 -0.02488483
  0.04205065  0.26708463  0.09729265  0.02450947  0.06092807 -0.0807007
  0.10640024 -0.26183122  0.13263775  0.09695752  0.07346341  0.06595689
  0.02631989 -0.14619662  0.00823605  0.10145645 -0.15708387  0.11319319
  0.0327839   0.04298275 -0.0804873  -0.00526643  0.30843231  0.20347202
 -0.09898984 -0.13062172  0.14360543 -0.12950534 -0.03596407  0.05742529
 -0.08864137 -0.19176076 -0.28356785  0.13511759  0.4323445   0.13189162
 -0.1895151   0.01497605 -0.07880417  0.00532003  0.

'ABDUL SAMAD'

In [4]:
##############################NOT DataBase###################
def timecomp(s):
    number = int(s.split(':')[0]+s.split(':')[1]+s.split(':')[2])
    return number
count=0
vcount = 0
enc_csv = pd.read_csv('encodingsKnown.csv')
Names = [n for n in enc_csv['Names']]
encodeKnown = []
for enc in range(len(enc_csv)):
    
    encodeKnown.append(np.array([float(i) for i in enc_csv['Encodings'][enc].replace('[','').replace(']','').replace(',','').split()]))

cap = cv2.VideoCapture(0)
attend = pd.read_csv('attendance.csv')
in_out = pd.read_csv('in_out.csv')


current_in={}     #Employees currently in office
entered = 0       #Variable used to acknowledge the updated file when 1
current=0

while True:
    if current==0:
        with open("file.txt", "r") as file: 
            data = file.readlines() 
            for line in data: 
                word = line.split() 
                #print (word) 
        current_in = {}
        for i in data:
            current_in[i.split()[0]]=i.split()[1] 
        current=1
    if entered==1:        #Save file when var=1
        in_out = pd.read_csv('in_out.csv') 
        entered=0
           
    _, img = cap.read()
    image = cv2.resize(img,(0,0),None,0.25,0.25)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    facesInFrame = face_recognition.face_locations(image)
    encodesInFrame = face_recognition.face_encodings(image,facesInFrame)

    for encFace,faceLoc in zip(encodesInFrame,facesInFrame):
        #print(encFace,faceLoc)
        matchList = face_recognition.compare_faces(encodeKnown,encFace)
       # print(matchList)
        faceDis = face_recognition.face_distance(encodeKnown,encFace)
        print(faceDis)
        match1 = min(faceDis)          #min Distance 
        #match = faceDis.index(match1)
        match = np.argmin(faceDis)     #index of min distance 

        
        
        if match1<0.5:
            
            if match1<=0.40:
                
                count+=1
                vcount=0
                if count>=3:
                    name = Names[match].upper()
#                     try:
#                         playsound('D:/Audio/%s.mp3'%(name))
#                     except:
#                         print(name)
#                     playsound('D:/59089586.mp3')
                    count = 0
                    
                    y,w,z,x = faceLoc
                    y, w, z, x1 = y*4,w*4,z*4,x*4
                    cv2.rectangle(img,(x,y),(w,z),(0,255,0),2)
                    cv2.rectangle(img,(x,z-35),(w,z),(0,255,0),cv2.FILLED)
                    cv2.putText(img,name,(x+6,z-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                    print(name)

                    curr = datetime.now()
                    dt = curr.strftime('%d-%m-%Y')
                    time = curr.strftime('%H:%M:%S')
                    itime = timecomp(time)
                    if name not in current_in:
                        if name not in [q for q in in_out[(in_out['Name']==name) & (in_out['Date']==dt)]['Name']]:
                            current_in[name]=itime
                            in_out.loc[len(in_out.index)]=[name,dt,time,nan,nan]
                            entered = 1
                            f1 = open('file.txt','a')
                            f1.write(name+' '+time+'\n')
                            f1.close()
                            in_out.to_csv('in_out.csv',index=False)
                            current = 0
                        else:
                            try:
                                last_out = timecomp([j for j in in_out[(in_out['Name']==name) & (in_out['Date']==dt) & (in_out['logout_time'].isna()==False)]['logout_time']][-1])
                                if last_out+150<itime:
                                    f1 = open('file.txt','a')
                                    f1.write(name+' '+itime+'\n')
                                    f1.close()
                                    current_in[name]=itime
                                    in_out.loc[len(in_out.index)]=[name,dt,time,nan,nan]
                                    entered = 1
                                    in_out.to_csv('in_out.csv',index=False)
                                    current=0
                            except:
                                print('in_pass')
                                pass
                    else:
                        try:
                            if name in [j for j in in_out[(in_out['Name']==name) & (in_out['Date']==dt) & (in_out['logout_time'].isna()==True)]['Name']]:
                                last_in = [k for k in in_out[(in_out['Name']==name) & (in_out['Date']==dt) & (in_out['logout_time'].isna()==True)]['login_time']][-1]
                                if timecomp(last_in)+500<itime:
                                    worktime = str(int(time[:2])-int(str(last_in)[:2]))+':'+str(int(time[3:5])-int(str(last_in)[3:5]))+':'+str(int(time[6:7])-int(str(last_in)[6:7]))
                                    indx = in_out[(in_out['Name']==name) & (in_out['Date']==dt) & (in_out['logout_time'].isna()==True)].index[0]
                                    in_out.drop(indx,axis=0,inplace=True)
                                    in_out.loc[len(in_out.index)]=[name,dt,timecomp(last_in),time,worktime]
                            
                            #write current_in file 
                                    try:
                                        for i in range(len(data)):
                                            if name in data[i]:
                                                data.pop(i)
                                    except:
                                        print('Current_in', data,'::::::::::::::::::::::::::::::::::::::::::')
                                    f1 = open('file.txt','a')
                                    for i in data:
                                        f1.write(i)
                                    f1.close()
                                    
                                    current=0
                                    current_in.pop(name)
                                    entered = 1
                                    in_out.to_csv('in_out.csv',index=False)
                        except:
                            print('OUT_Error')
                            pass
        else:
            vcount+=1
            count=0
            if vcount>=6:
                y,w,z,x = faceLoc
                y, w, z, x1 = y*4,w*4,z*4,x*4
                cv2.rectangle(img,(x,y),(w,z),(0,255,0),2)
                cv2.rectangle(img,(x,z-35),(w,z),(0,255,0),cv2.FILLED)
                cv2.putText(img,'visitor',(x+6,z-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
               # Attendance('Visitor')
                print('Visitor')
                vcount=0
                
    cv2.imshow('Webcam',img)
    k = cv2.waitKey(1)
    if k == 27:         # wait for ESC key to exit
        break
    cv2.waitKey(1)
cv2.destroyAllWindows()

[0.7017357  0.73106213 0.73413216 0.56074329 0.56074329 0.63179207
 0.62643702 0.6122753  0.650671   0.66954551 0.6610752  0.6283682
 0.62723185 0.59679335 0.65079684 0.67387964 0.69478735 0.64948404
 0.68834579 0.59588447 0.59770593 0.57585283 0.75854977 0.75854977
 0.71850465 0.7240516  0.74230129 0.731342   0.70360584 0.56339074
 0.58334461 0.56532616 0.659429   0.62366966 0.63797461 0.49939615]
[0.63984119 0.65368945 0.64971808 0.57714662 0.57714662 0.6073628
 0.64553899 0.69257716 0.62106506 0.63526221 0.61680576 0.61871836
 0.53296663 0.56745857 0.61553639 0.58511533 0.60520455 0.62049125
 0.62825454 0.63312935 0.67074967 0.68236541 0.57425931 0.57425931
 0.57658228 0.59596834 0.57257693 0.61438777 0.55880643 0.77380302
 0.78422773 0.78784629 0.59729297 0.60777925 0.57261164 0.69579741]
[0.7317169  0.75710907 0.75019547 0.57138552 0.57138552 0.63750257
 0.64851196 0.61722398 0.67087073 0.68682144 0.66950513 0.6340527
 0.65177026 0.63782349 0.6903273  0.66548436 0.70077905 0.67327

[0.68327326 0.71389475 0.68096477 0.56250306 0.56250306 0.63635797
 0.58968181 0.55788869 0.61905253 0.62249966 0.67085854 0.62561355
 0.60716777 0.61147793 0.67579679 0.61790151 0.62619456 0.65363904
 0.67162772 0.58298131 0.57991517 0.55707348 0.73749025 0.73749025
 0.68325509 0.67392891 0.65874405 0.71122314 0.66401616 0.59175378
 0.59735393 0.61825802 0.66919898 0.63540658 0.63536852 0.42182398]
[0.74333848 0.77796033 0.77603796 0.62823231 0.62823231 0.69035631
 0.71036571 0.70819575 0.71177428 0.69132454 0.7680663  0.73210733
 0.71149466 0.69016303 0.7342269  0.72203925 0.74187019 0.73175558
 0.75803088 0.58157298 0.61450128 0.62489479 0.78987794 0.78987794
 0.76118476 0.75152276 0.75578304 0.75580992 0.72014371 0.60061478
 0.62800439 0.6083959  0.70771159 0.67465737 0.67075364 0.46393765]
[0.73473765 0.76743743 0.75204897 0.59361818 0.59361818 0.67260422
 0.59598442 0.59622944 0.62416909 0.64094572 0.68164129 0.61935289
 0.62746365 0.62779664 0.69045707 0.6963749  0.70982605 0.68

[0.71560936 0.72766072 0.73210046 0.58343699 0.58343699 0.66014925
 0.63150899 0.62014192 0.63833951 0.64457042 0.70359935 0.6726891
 0.69511222 0.70316096 0.74319388 0.69616177 0.69755448 0.73315157
 0.74879611 0.6096107  0.60993154 0.58316729 0.80281694 0.80281694
 0.73193519 0.74901835 0.77009584 0.77856017 0.75751327 0.57936388
 0.61827365 0.61534649 0.73772392 0.70166015 0.65983094 0.38716488]
ABDUL SAMAD
in_pass
[0.72559133 0.74381274 0.74816464 0.56832918 0.56832918 0.66255143
 0.6518714  0.63631161 0.66466668 0.67480205 0.71145545 0.69719522
 0.71017925 0.69709498 0.74832008 0.71751559 0.71105871 0.70052853
 0.72420428 0.63222146 0.64594622 0.6114018  0.77270248 0.77270248
 0.72510846 0.74435605 0.77156866 0.76988801 0.74395897 0.61412068
 0.64895159 0.64997296 0.67279668 0.63879379 0.62750373 0.40715682]
[0.70593493 0.72344641 0.74218559 0.52351079 0.52351079 0.63435272
 0.62162511 0.61571028 0.64568408 0.65530606 0.68920469 0.66268924
 0.66415652 0.63710043 0.70231392 0.69096

[0.69006473 0.69846207 0.69600566 0.5803866  0.5803866  0.65969478
 0.61641694 0.59627911 0.65431704 0.66624769 0.66490038 0.6434382
 0.67608728 0.64468451 0.70113414 0.67368624 0.66701352 0.66164659
 0.68088015 0.58381843 0.58191255 0.53739439 0.76553736 0.76553736
 0.71790357 0.71555542 0.74526329 0.74813602 0.72679864 0.58589287
 0.600132   0.61538569 0.68177885 0.64970759 0.62858976 0.37607289]
[0.73468573 0.7303212  0.74937651 0.64810908 0.64810908 0.64612308
 0.74558159 0.79695917 0.63081593 0.66475848 0.6246405  0.6753943
 0.64934016 0.66874264 0.67690302 0.64896588 0.73357069 0.62791682
 0.64249063 0.72660766 0.77237296 0.81600055 0.70299609 0.70299609
 0.67094022 0.68982393 0.68531685 0.65057128 0.65301945 0.71463227
 0.72027093 0.71224737 0.71394041 0.70218839 0.76176755 0.83906906]
[0.6869382  0.70429771 0.70615266 0.54807661 0.54807661 0.64497693
 0.61253087 0.61236254 0.61590669 0.63044102 0.64131732 0.61187052
 0.69040929 0.65508417 0.70860436 0.65059174 0.65107775 0.6441

[0.70094551 0.72129977 0.73521518 0.55765852 0.55765852 0.68034781
 0.6317794  0.62556631 0.6486063  0.66473049 0.69720847 0.67699785
 0.67366977 0.6517413  0.70689516 0.69725387 0.69251678 0.69680732
 0.73435178 0.58191327 0.59237405 0.56088054 0.76313767 0.76313767
 0.71048342 0.73529774 0.74907452 0.76940268 0.73539508 0.6074678
 0.6205959  0.63882193 0.68939957 0.66866598 0.63035175 0.43243954]
[0.66024093 0.64540839 0.66041241 0.59765766 0.59765766 0.60470955
 0.67685438 0.73060683 0.60293957 0.61614862 0.56746655 0.6255175
 0.5874187  0.60387019 0.60989727 0.57265888 0.62373551 0.61995405
 0.63055171 0.65332408 0.70382079 0.72955556 0.64987877 0.64987877
 0.63106636 0.60064262 0.60148648 0.59556737 0.5673917  0.76427513
 0.76273235 0.75595091 0.61279812 0.61321616 0.71306646 0.78638608]
[0.65457738 0.65726678 0.66953949 0.5364216  0.5364216  0.65468396
 0.64443805 0.60935451 0.64702474 0.64400364 0.6522151  0.6533359
 0.70275867 0.66448721 0.71534625 0.67331684 0.6647295  0.67749

...

[0.58717885 0.61343503 0.62625804 0.49437523 0.49437523 0.54946943
 0.57261741 0.62805144 0.59158372 0.61787269 0.52464636 0.54375341
 0.50247618 0.47826343 0.54432071 0.58143236 0.61781883 0.56147253
 0.57452855 0.60600527 0.6195304  0.65347249 0.56669737 0.56669737
 0.55354251 0.58751786 0.56782515 0.58083239 0.56059017 0.71302949
 0.70956415 0.71863844 0.55941727 0.56799283 0.56398475 0.68135639]
[0.62180022 0.63772916 0.65957248 0.53563371 0.53563371 0.58697597
 0.60275565 0.66412501 0.61983037 0.63549169 0.52660019 0.54598597
 0.54449514 0.5254035  0.56786129 0.60208404 0.64044389 0.61258339
 0.62773725 0.62432432 0.64188757 0.68531485 0.58355325 0.58355325
 0.55615717 0.62503771 0.61194282 0.62253378 0.59205281 0.73866338
 0.74186514 0.74050778 0.5712231  0.58305109 0.61071418 0.70849704]
[0.59871027 0.62112256 0.63355744 0.55070403 0.55070403 0.60511481
 0.58355048 0.64512049 0.59159137 0.60260105 0.51075506 0.54422856
 0.49734522 0.50159836 0.56143966 0.59277056 0.62057501 0.59

[0.61526305 0.62545541 0.62056412 0.56051352 0.56051352 0.59839625
 0.5650311  0.60364893 0.5580178  0.59241055 0.57214904 0.6069383
 0.45912234 0.51128731 0.55479203 0.5734837  0.60272347 0.61273823
 0.62614287 0.65065314 0.66667226 0.6936651  0.62812696 0.62812696
 0.59165493 0.58632951 0.54953592 0.62037574 0.58831408 0.73934755
 0.73250096 0.76027591 0.66363331 0.67504013 0.64263311 0.72823026]
[0.61329612 0.63337808 0.62252513 0.57564326 0.57564326 0.59681467
 0.5646936  0.61384284 0.56950605 0.5953275  0.59654099 0.63401695
 0.47864731 0.52723091 0.56522317 0.58335158 0.59912369 0.60432554
 0.6200687  0.64817302 0.67329579 0.70684903 0.61857526 0.61857526
 0.58857233 0.56668682 0.54642348 0.60102713 0.5746772  0.7648746
 0.76033825 0.78563572 0.62811088 0.64354369 0.64801821 0.73386379]
[0.57059747 0.58733375 0.57123067 0.54063499 0.54063499 0.57473917
 0.52933395 0.56728336 0.51493192 0.53750174 0.55738259 0.59821251
 0.47815529 0.50665211 0.56250509 0.5407018  0.55766571 0.5584

[0.58154027 0.61103506 0.61336463 0.53175541 0.53175541 0.58757529
 0.54656835 0.60007304 0.54668618 0.5667254  0.55711359 0.57477616
 0.43057237 0.43762536 0.51388011 0.53730238 0.54169401 0.56426501
 0.57543454 0.57380236 0.61426066 0.63813965 0.56100009 0.56100009
 0.55435241 0.5484954  0.5077436  0.56217941 0.51388025 0.71640359
 0.71688869 0.72790831 0.58155985 0.58918847 0.58489452 0.67652963]
[0.6089004  0.63451401 0.62923604 0.54780396 0.54780396 0.58753906
 0.5656337  0.61575341 0.55855503 0.5776435  0.55362108 0.57839316
 0.41890924 0.44701453 0.50931822 0.55793786 0.56823851 0.58204147
 0.59121408 0.61313062 0.65021278 0.67377139 0.55172642 0.55172642
 0.54085987 0.57112664 0.52661137 0.58060148 0.53464233 0.74781841
 0.75187265 0.75717382 0.58506711 0.595072   0.62205318 0.71021658]
[0.61854338 0.64960225 0.63835998 0.56113177 0.56113177 0.59831741
 0.57667637 0.62641142 0.56057144 0.56969648 0.57133119 0.58999631
 0.42130659 0.47386475 0.5512107  0.59564081 0.5933823  0.61

[0.61259062 0.64687987 0.64622479 0.56143526 0.56143526 0.58843074
 0.61556017 0.64176068 0.60601172 0.6490816  0.51299499 0.56163785
 0.49294794 0.51451696 0.55493694 0.56750205 0.5955515  0.56959658
 0.58326981 0.68184062 0.70710605 0.72154883 0.59868554 0.59868554
 0.58405791 0.61561315 0.56472806 0.6078629  0.58993066 0.7471546
 0.74465053 0.75701127 0.60885096 0.62227807 0.69784825 0.75450455]
[0.59167493 0.63890113 0.62977703 0.55545692 0.55545692 0.5887083
 0.61368226 0.65702499 0.59754209 0.63152887 0.55389784 0.58771961
 0.47084009 0.48225343 0.53540015 0.59112044 0.61025932 0.57729101
 0.58394554 0.64764875 0.67768178 0.70431713 0.57100147 0.57100147
 0.55731318 0.61123833 0.57937115 0.59831584 0.58163859 0.76815061
 0.7655164  0.77541008 0.59471258 0.60768084 0.66250109 0.7356206 ]
[0.58160484 0.62697782 0.62457472 0.53789176 0.53789176 0.58870846
 0.61898757 0.64248198 0.61386742 0.63261177 0.4991534  0.54072148
 0.48208991 0.45919901 0.51260882 0.56540323 0.59740935 0.5647

KeyboardInterrupt: 